In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
import os

ROOT_PATH = "/content/drive/MyDrive/ADA/"
print(os.listdir(ROOT_PATH)) # Check the content of the path
os.chdir(ROOT_PATH) # cd into directory
print(os.listdir(".")) # Check the content of current folder

['book_film_summaries.csv', 'bookfilm_summaries_with_similarity.csv']
['book_film_summaries.csv', 'bookfilm_summaries_with_similarity.csv']


In [5]:
!pip install langchain-core langchain-community -U sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.4
    Uninstalling langchain-0.3.4:
      Successfully uninstalled langchain-0.3.4


In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import AutoTokenizer, AutoModelForMaskedLM
from sentence_transformers import SentenceTransformer
from langchain_core.embeddings import Embeddings
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import torch

In [7]:
class CustomEmbeddingsChunking(Embeddings):
    def __init__(self, model):
        self.model = model
        self.max_length = model.max_seq_length

    def chunk_text(self, text, max_tokens, overlap=50):
      words = text.split()
      chunks = []
      start = 0
      while start < len(words):
          end = min(start + max_tokens, len(words))
          chunk = ' '.join(words[start:end])
          chunks.append(chunk)
          if end == len(words):
              break
          start = end - overlap  # Move start forward with overlap
      return chunks

    def embed_documents(self, texts, batch_size=32):
        ''' Embed a list of texts and return the embeddings '''
        embeddings = []
        for i in range(0, len(texts)):

            batch_texts = texts[i]
            doc_embedding = self.model.encode(batch_texts, batch_size=batch_size, show_progress_bar=False)
            embeddings.append(doc_embedding)

        return embeddings

    def embed_query(self, text):
        return self.model.encode([text])[0]

    def embed_text_pair(self, text_1, text_2):
        ''' Embed two texts and return their embeddings '''
        chunks_1 = self.chunk_text(text_1, self.max_length)
        chunks_2 = self.chunk_text(text_2, self.max_length)

        return np.mean(self.embed_documents(chunks_1), axis=0), np.mean(self.embed_documents(chunks_2), axis=0)

## Load Model

In [8]:
model = SentenceTransformer('all-mpnet-base-v2')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


from transformers import AutoTokenizer, AutoModelForMaskedLM

sentiment_tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-large")
sentiment_model = AutoModelForMaskedLM.from_pretrained("FacebookAI/xlm-roberta-large")
sentiment_model = sentiment_model.to(device)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at FacebookAI/xlm-roberta-large were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Load Data

In [13]:
summaries_df = pd.read_csv('./book_film_summaries.csv')
summaries_df = summaries_df.drop('Unnamed: 0', axis=1)


summaries_df.head()

,movie_id,book_title,movie_name,book_summary,film_summary
0,196176,The Siege of Trencher's Farm,Straw Dogs,"George Magruder, an American professor of Eng...","David Sumner , a timid American mathematician,..."
1,6703617,Journey to the West,The Forbidden Kingdom,The novel has 100 chapters. These can be divi...,"In this film, which is based loosely on the an..."
2,24416479,The Beast Master,Beastmaster 2: Through the Portal of Time,"It tells the story of Hosteen Storm, an ex-so...","Dar, the Beastmaster , is back and now he has ..."
3,9384481,The Blessing,Count Your Blessings,It is set in the post-war World War II period...,While visiting Grace Allingham in wartime Lond...
4,22224559,The Last Song,The Last Song,Veronica “Ronnie” Miller’s life was turned up...,"At seventeen, Veronica ""Ronnie"" Miller remain..."


## Run NLP Analysis

In [14]:
def cosine_similarity_from_text_pair(text_1, text_2):
    embedding_1, embedding_2 = embeddings.embed_text_pair(text_1, text_2)
    return cosine_similarity(embedding_1.reshape(1, -1), embedding_2.reshape(1, -1))[0][0]

def sentiment_of_text(text):
    print("implement this...")

In [15]:
embeddings = CustomEmbeddingsChunking(model)

In [16]:
tqdm.pandas()

summaries_df['similarity'] = summaries_df.progress_apply(lambda row: cosine_similarity_from_text_pair(row['film_summary'], row['book_summary']), axis=1)
summaries_df.head()

100%|██████████| 840/840 [01:42<00:00,  8.20it/s]


,movie_id,book_title,movie_name,book_summary,film_summary,similarity
0,196176,The Siege of Trencher's Farm,Straw Dogs,"George Magruder, an American professor of Eng...","David Sumner , a timid American mathematician,...",0.451876
1,6703617,Journey to the West,The Forbidden Kingdom,The novel has 100 chapters. These can be divi...,"In this film, which is based loosely on the an...",0.559595
2,24416479,The Beast Master,Beastmaster 2: Through the Portal of Time,"It tells the story of Hosteen Storm, an ex-so...","Dar, the Beastmaster , is back and now he has ...",0.344548
3,9384481,The Blessing,Count Your Blessings,It is set in the post-war World War II period...,While visiting Grace Allingham in wartime Lond...,0.669707
4,22224559,The Last Song,The Last Song,Veronica “Ronnie” Miller’s life was turned up...,"At seventeen, Veronica ""Ronnie"" Miller remain...",0.852735


In [19]:
summaries_df.to_csv('bookfilm_summaries_with_similarity.csv', index=False)

In [18]:
film_lengths = summaries_df['film_summary'].apply(lambda film: len(film.split())).tolist()

# Calculate the lengths of book summaries
book_lengths = summaries_df['book_summary'].apply(lambda book: len(book.split())).tolist()


print(max(film_lengths))
print(max(book_lengths))

2484
4719
